In [72]:
import networkx as nx
import pandas as pd
from pyvis.network import Network

In [73]:
data_file = "data/calci_bollywood_movies.csv"

In [74]:
df = pd.read_csv(data_file,delimiter="\t")
print(df.columns)

Index(['imdbId', 'title', 'releaseYear', 'releaseDate', 'genre', 'writers',
       'actors', 'directors', 'sequel', 'hitFlop'],
      dtype='object')


data is very big for now, let's sample only 500 records

In [75]:
df = df.sample(n=50,replace=False)

Let's take only movie tile and writers  

In [77]:
movie_writers = []
for index in df.index:
    title = df["title"][index].lower()
    all_writers = df["writers"][index]
    if isinstance(all_writers,str):
        writers = all_writers.split("|")
    for s in writers:
        if isinstance(s,str):
            role = s[s.find("(")+1:s.find(")")].lower().strip()
            name = s.replace(role,"").replace("(","").replace(")","").lower().strip()
            print("%s ----%s BY------%s"%(title,role,name))
            movie_writers.append([title,name,role])

aamdani atthanni kharcha rupaiya ----dialogue BY------anwar khan
aamdani atthanni kharcha rupaiya ----story BY------v. shekhar
udaan ----vikramaditya motwane BY------vikramaditya motwane
udaan ----anurag kashyap BY------anurag kashyap
udaan ----rohan's poems and stories BY------satyanshu singh rohan's poems and stories
udaan ----rohan's poems and stories BY------devanshu singh rohan's poems and stories
kai po che! ----novel BY------chetan bhagat
kai po che! ----adaptation BY------abhishek kapoor
kai po che! ----adaptation BY------supratik sen
kai po che! ----adaptation BY------chetan bhagat
kai po che! ----adaptation BY------pubali chaudhuri
chal chala chal ----lyrics BY------anand raj anand
chal chala chal ----lyrics BY------sunil jha
chal chala chal ----dialogue BY------manisha korde
chal chala chal ----lyrics BY------vaibhav modi
my friend ganesha 2 ----story BY------rajiv s. ruia
my friend ganesha 2 ----s. sachinde BY------s. sachinder
badhaai ho badhaai ----kavita choudhar BY-----

Now make a graph with writers and titles where the relation is the role

In [78]:
movie_graph = Network()
for item in movie_writers:
    t = item[0]
    a = item[1]
    r = item[2]
    attrs = {t:{"type":"title"},a:{"type":"writer"}}
    movie_graph.add_node(t,color="red",label=t)
    movie_graph.add_node(a,color="blue",label=a)
    movie_graph.add_edge(t,a,label=r)
    # nx.set_node_attributes(movie_graph,attrs)

Render the graph, see the relations

In [79]:
# nt = Network(directed=False)
# nt.from_nx(movie_graph)
movie_graph.show("data/movie_2node_graph.html",notebook=False )

data/movie_2node_graph.html


Now let's include actors as well

In [80]:
movie_writer_actor = []
for index in df.index:
    title = df["title"][index].lower()
    all_writers = df["writers"][index]
    if isinstance(all_writers,str):
        writers = all_writers.split("|")
    for s in writers:
        if isinstance(s,str):
            role = s[s.find("(")+1:s.find(")")].lower().strip()
            name = s.replace(role,"").replace("(","").replace(")","").lower().strip()
            print("%s ----%s BY------%s"%(title,role,name))
            movie_writer_actor.append([title,name,role])
    all_actors = df["actors"][index]
    if isinstance(all_actors,str):
        actors = all_actors.split("|")
    for a in actors:
        movie_writer_actor.append([title,a,"actor"])

aamdani atthanni kharcha rupaiya ----dialogue BY------anwar khan
aamdani atthanni kharcha rupaiya ----story BY------v. shekhar
udaan ----vikramaditya motwane BY------vikramaditya motwane
udaan ----anurag kashyap BY------anurag kashyap
udaan ----rohan's poems and stories BY------satyanshu singh rohan's poems and stories
udaan ----rohan's poems and stories BY------devanshu singh rohan's poems and stories
kai po che! ----novel BY------chetan bhagat
kai po che! ----adaptation BY------abhishek kapoor
kai po che! ----adaptation BY------supratik sen
kai po che! ----adaptation BY------chetan bhagat
kai po che! ----adaptation BY------pubali chaudhuri
chal chala chal ----lyrics BY------anand raj anand
chal chala chal ----lyrics BY------sunil jha
chal chala chal ----dialogue BY------manisha korde
chal chala chal ----lyrics BY------vaibhav modi
my friend ganesha 2 ----story BY------rajiv s. ruia
my friend ganesha 2 ----s. sachinde BY------s. sachinder
badhaai ho badhaai ----kavita choudhar BY-----

Recreate Graph with new nodes

In [81]:
movie_graph = Network()
for item in movie_writer_actor:
    t = item[0]
    a = item[1]
    r = item[2]
    movie_graph.add_node(t,color="red",label=t,group="title") # add title
    if r == "actor":
        movie_graph.add_node(a,color="green",label=a,group="actor")
        label = "actor"
    else:
        movie_graph.add_node(a,color="blue",label=a,group="writer")
        label = r
    movie_graph.add_edge(t,a,label=label)
    # nx.set_node_attributes(movie_graph,attrs)

Show the graph now

In [82]:
movie_graph.show("data/movie_3node_graph.html",notebook=False )

data/movie_3node_graph.html


In [59]:
nx_graph = nx.Graph()
edges = movie_graph.get_edges()
for e in edges:
    n1 = e["from"]
    n1_info = movie_graph.get_node(n1)
    n1_group = n1_info["group"]
    n2 = e["to"]
    n2_info = movie_graph.get_node(n2)
    n2_group = n2_info["group"]
    n1_name = "%s_%s"%(n1,n1_group)
    n2_name = "%s_%s"%(n2,n2_group)
    # print(n1_name,n2_name)
    nx_graph.add_edge(n1_name,n2_name)
print("Number of nodes : ",nx_graph.number_of_nodes())
print("Number of nodes : ",nx_graph.number_of_edges())

Number of nodes :  910
Number of nodes :  963


In [61]:
import community as community_louvain

In [63]:
partition = community_louvain.best_partition(nx_graph)

In [66]:
communities = {}
for k,v in partition.items():
    if v not in communities.keys():
        communities[v] = [k]
    else:
        communities[v].append(k)

Randomly pick a community to check

In [71]:
community_id = 1
members = communities[community_id]
for m in members:
    if "title" in m:
        print(m)

cheeni kum_title
manorama six feet under_title
anubhav: an actor's tale_title
allah ke banday_title
daman: a victim of marital violence_title
agyaat: the unknown_title
department_title
